# SOC-Level Anomaly Detection (Advanced)
### Scenarios: multi-stage brute force + impossible travel

You will build **SOC-style detections** and produce a concise incident narrative.


In [ ]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, asin, sqrt
from datetime import timedelta


In [ ]:
df = pd.read_csv('soc_auth_activity_advanced.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)
df.head()

## 1) Detection A – Multi-Stage Brute Force Chain

In [ ]:
# Windowed counts of AUTH_FAIL per (user, ip) over 10 minutes
fail = df[df['event_type'] == 'AUTH_FAIL'].copy().set_index('timestamp')
counts = fail.groupby(['user','ip_address']).resample('10min').size().reset_index(name='auth_fail_10m')
susp_fail = counts[counts['auth_fail_10m'] >= 25]
susp_fail.head()

In [ ]:
# TODO: For each suspicious window, check if AUTH_SUCCESS occurs soon after
# Build an alerts table with evidence.
alerts = []

def add(alert_type, severity, user, ip, start, end, evidence):
    alerts.append({
        'alert_type': alert_type,
        'severity': severity,
        'user': user,
        'ip_address': ip,
        'start_time': start,
        'end_time': end,
        'evidence': evidence
    })

for _, r in susp_fail.iterrows():
    user, ip, st = r['user'], r['ip_address'], r['timestamp']
    window_end = st + pd.Timedelta('10min')
    # Look 30 minutes after for success
    after = df[(df['user']==user) & (df['ip_address']==ip) & (df['timestamp']>=st) & (df['timestamp']<=st+pd.Timedelta('40min'))]
    if (after['event_type']=='AUTH_SUCCESS').any():
        add('BRUTE_FORCE_THEN_SUCCESS','CRITICAL',user,ip,st,window_end,
            evidence=f"{int(r['auth_fail_10m'])} AUTH_FAIL in 10m, then AUTH_SUCCESS within 40m")

pd.DataFrame(alerts).head()

## 2) Detection B – Privilege Escalation After Suspicious Auth

In [ ]:
# TODO: If PRIV_ESCALATION occurs within 30 minutes after a CRITICAL auth alert, flag it.
pass

## 3) Detection C – Impossible Travel

In [ ]:
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2-lat1)
    dl = radians(lon2-lon1)
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dl/2)**2
    return 2*R*asin(sqrt(a))

# Compute consecutive successful auths per user and estimate travel speed
auth = df[df['event_type']=='AUTH_SUCCESS'].copy().sort_values(['user','timestamp'])
auth['prev_time'] = auth.groupby('user')['timestamp'].shift(1)
auth['prev_lat'] = auth.groupby('user')['lat'].shift(1)
auth['prev_lon'] = auth.groupby('user')['lon'].shift(1)
auth['prev_loc'] = auth.groupby('user')['location'].shift(1)

mask = auth['prev_time'].notna()
auth2 = auth[mask].copy()
auth2['dt_hours'] = (auth2['timestamp'] - auth2['prev_time']).dt.total_seconds() / 3600.0
auth2['dist_km'] = auth2.apply(lambda r: haversine_km(r['prev_lat'], r['prev_lon'], r['lat'], r['lon']), axis=1)
auth2['speed_kmh'] = auth2['dist_km'] / auth2['dt_hours'].replace(0, np.nan)

# Flag if speed exceeds 900 km/h (faster than realistic travel between logins)
impossible = auth2[auth2['speed_kmh'] > 900].copy()
impossible[['user','prev_time','prev_loc','timestamp','location','dist_km','dt_hours','speed_kmh']].head(10)

In [ ]:
# Convert impossible travel rows into alerts
for _, r in impossible.iterrows():
    add('IMPOSSIBLE_TRAVEL','HIGH', r['user'], r['ip_address'], r['prev_time'], r['timestamp'],
        evidence=f"{r['prev_loc']} -> {r['location']} in {r['dt_hours']:.2f}h (~{r['speed_kmh']:.0f} km/h)")

alerts_df = pd.DataFrame(alerts)
alerts_df.sort_values(['severity','start_time']).head(20)

## 4) Detection D – Data Exfiltration (bytes_out outliers)

In [ ]:
# Simple baseline: per-user bytes_out distribution
exports = df[df['event_type'].isin(['DATA_EXPORT','FILE_READ'])].copy()
stats = exports.groupby('user')['bytes_out'].agg(['mean','std','count']).reset_index()
exports = exports.merge(stats, on='user', how='left')
exports['z'] = (exports['bytes_out'] - exports['mean']) / (exports['std'].replace(0, np.nan))
exfil = exports[(exports['event_type']=='DATA_EXPORT') & (exports['z'] > 3)].copy()
exfil[['timestamp','user','ip_address','bytes_out','z']].head(10)

In [ ]:
# TODO: Add DATA_EXFIL alerts to the table with severity HIGH/CRITICAL
pass

## 5) Export + SOC Narrative

In [ ]:
alerts_df = pd.DataFrame(alerts)
alerts_df.to_csv('soc_alerts.csv', index=False)
alerts_df

### SOC Analyst Narrative (required)
Write a short narrative (8–12 bullets):
1. **Timeline**: What happened first → next → last?
2. **Most severe alert** and why
3. **Evidence** (counts, windows, IPs, locations)
4. **Likely attack type** (e.g., credential stuffing, account takeover)
5. **Recommended response actions** (containment + investigation)
